In [ ]:
# Install Pytorch & other libraries
%pip install -q tensorboard wandb

# Install Hugging Face libraries
%pip install -q --upgrade \
  "transformers==4.45.1" \
  "datasets==3.0.1" \
  "accelerate==0.34.2" \
  "evaluate==0.4.3" \
  "bitsandbytes==0.44.0" \
  "trl==0.11.1" \
  "peft==0.13.0" \
  "qwen-vl-utils" 

%pip install "Pillow>=9.4.0"
%pip install scikit-learn

In [3]:
%pip uninstall -y transformers datasets accelerate evaluate bitandbytes trl peft qwen-vl-utils wandb

Note: you may need to restart the kernel to use updated packages.


In [8]:
# Install Hugging Face libraries
%pip install -q \
  "transformers==4.45.1" \
  "datasets==3.6.0" \
  "accelerate==1.8.1" \
  "evaluate==0.4.3" \
  "bitsandbytes==0.46.0" \
  "trl==0.20.0" \
  "peft==0.15.2" \
  "qwen-vl-utils==0.0.11" \
  "wandb==0.20.1"


ERROR: Cannot install transformers==4.45.1 and trl==0.20.0 because these package versions have conflicting dependencies.

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip
ERROR: ResolutionImpossible: for help visit https://pip.pypa.io/en/latest/topics/dependency-resolution/#dealing-with-dependency-conflicts
Note: you may need to restart the kernel to use updated packages.


In [9]:
!pip install  -U -q git+https://github.com/huggingface/transformers.git git+https://github.com/huggingface/trl.git datasets bitsandbytes peft qwen-vl-utils wandb accelerate


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [10]:
!pip install -q torch==2.4.1+cu121 torchvision==0.19.1+cu121 torchaudio==2.4.1+cu121 --extra-index-url https://download.pytorch.org/whl/cu121


[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [12]:
!pip install scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 88.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 35.4/35.4 MB 150.7 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 25.0.1 -> 25.2
[notice] To update, run: python -m pip install --upgrade pip


In [13]:
import io
import json
from PIL import Image
from datasets import load_dataset
from sklearn.model_selection import train_test_split
import torch
from transformers import AutoModelForVision2Seq, AutoProcessor
from trl import SFTConfig, SFTTrainer
from transformers import Qwen2VLProcessor
from qwen_vl_utils import process_vision_info
from peft import LoraConfig

In [14]:
import wandb
wandb.init(mode="disabled")

In [15]:
# 시스템(assistant)에게 주어진 역할
system_message = "당신은 이미지와 제품명(name)으로부터 패션/스타일 정보를 추론하는 분류 모델입니다."

# 실제로 사용자 입력 -> 모델이 답해야 하는 프롬프트
prompt = """입력 정보:
- name: {name}
- image: [image]

위 정보를 바탕으로, 아래 7가지 key에 대한 값을 JSON 형태로 추론해 주세요:
1) gender
2) masterCategory
3) subCategory
4) season
5) usage
6) baseColour
7) articleType

출력 시 **아래 JSON 예시 형태**를 반드시 지키세요:
{{
  "gender": "예시값",
  "masterCategory": "예시값",
  "subCategory": "예시값",
  "season": "예시값",
  "usage": "예시값",
  "baseColour": "예시값",
  "articleType": "예시값"
}}

# 예시
{{
  "gender": "Men",
  "masterCategory": "Accessories",
  "subCategory": "Eyewear",
  "season": "Winter",
  "usage": "Casual",
  "baseColour": "Blue",
  "articleType": "Sunglasses"
}}

# 주의
- 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.
"""

In [16]:
def combine_cols_to_label(example):
    # 실제 컬럼명에 맞게 수정
    label_dict = {
        "gender": example["gender"],
        "masterCategory": example["masterCategory"],
        "subCategory": example["subCategory"],
        "season": example["season"],
        "usage": example["usage"],
        "baseColour": example["baseColour"],
        "articleType": example["articleType"],
    }
    example["label"] = json.dumps(label_dict, ensure_ascii=False)
    return example

def format_data(sample):
   # Image.Image를 PngImageFile로 변환
   buffer = io.BytesIO()
   sample["image"].save(buffer, format='PNG')
   buffer.seek(0)
   image = Image.open(buffer)

   return {
       "messages": [
           {
               "role": "system",
               "content": [
                   {
                       "type": "text",
                       "text": system_message
                   }
               ],
           },
           {
               "role": "user",
               "content": [
                   {
                       "type": "text",
                       "text": prompt.format(name=sample["productDisplayName"]),
                   },
                   {
                       "type": "image",
                       "image": image,
                   }
               ],
           },
           {
               "role": "assistant",
               "content": [
                   {
                       "type": "text",
                       "text": sample["label"],
                   }
               ],
           },
       ],
   }

In [17]:
dataset = load_dataset("ashraq/fashion-product-images-small", split="train")
dataset_add_label = dataset.map(combine_cols_to_label)
dataset_add_label = dataset_add_label.shuffle(seed=4242)

README.md:   0%|          | 0.00/867 [00:00<?, ?B/s]

(…)-00000-of-00002-6cff4c59f91661c3.parquet:   0%|          | 0.00/136M [00:00<?, ?B/s]

(…)-00001-of-00002-bb459e5ac5f01e71.parquet:   0%|          | 0.00/135M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/44072 [00:00<?, ? examples/s]

Map:   0%|          | 0/44072 [00:00<?, ? examples/s]

In [18]:
dataset_add_label[0]

{'id': 15516,
 'gender': 'Men',
 'masterCategory': 'Footwear',
 'subCategory': 'Flip Flops',
 'articleType': 'Flip Flops',
 'baseColour': 'Navy Blue',
 'season': 'Fall',
 'year': 2011.0,
 'usage': 'Casual',
 'productDisplayName': 'Rockport Men Altrezlp Navy Blue Flip Flops',
 'image': <PIL.Image.Image image mode=RGB size=60x80>,
 'label': '{"gender": "Men", "masterCategory": "Footwear", "subCategory": "Flip Flops", "season": "Fall", "usage": "Casual", "baseColour": "Navy Blue", "articleType": "Flip Flops"}'}

In [19]:
formatted_dataset = [format_data(row) for row in dataset_add_label]

In [20]:
formatted_dataset[0]

{'messages': [{'role': 'system',
   'content': [{'type': 'text',
     'text': '당신은 이미지와 제품명(name)으로부터 패션/스타일 정보를 추론하는 분류 모델입니다.'}]},
  {'role': 'user',
   'content': [{'type': 'text',
     'text': '입력 정보:\n- name: Rockport Men Altrezlp Navy Blue Flip Flops\n- image: [image]\n\n위 정보를 바탕으로, 아래 7가지 key에 대한 값을 JSON 형태로 추론해 주세요:\n1) gender\n2) masterCategory\n3) subCategory\n4) season\n5) usage\n6) baseColour\n7) articleType\n\n출력 시 **아래 JSON 예시 형태**를 반드시 지키세요:\n{\n  "gender": "예시값",\n  "masterCategory": "예시값",\n  "subCategory": "예시값",\n  "season": "예시값",\n  "usage": "예시값",\n  "baseColour": "예시값",\n  "articleType": "예시값"\n}\n\n# 예시\n{\n  "gender": "Men",\n  "masterCategory": "Accessories",\n  "subCategory": "Eyewear",\n  "season": "Winter",\n  "usage": "Casual",\n  "baseColour": "Blue",\n  "articleType": "Sunglasses"\n}\n\n# 주의\n- 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.\n'},
    {'type': 'image',
     'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=60x80>}]},
  {'role': 'assista

In [21]:
# test_size=0.9로 설정하여 전체 데이터의 90%를 테스트 세트로 분리
train_dataset, test_dataset = train_test_split(formatted_dataset,
                                             test_size=0.9,
                                             random_state=42)

In [22]:
print('학습 데이터의 개수:', len(train_dataset))
print('테스트 데이터의 개수:', len(test_dataset))

학습 데이터의 개수: 4407
테스트 데이터의 개수: 39665


In [23]:
# 허깅페이스 모델 ID
model_id = "Qwen/Qwen2-VL-7B-Instruct"

# 모델과 프로세서 로드
model = AutoModelForVision2Seq.from_pretrained(
   model_id,
   device_map="auto",                            # GPU 메모리에 자동 할당
   torch_dtype=torch.bfloat16,                   # bfloat16 정밀도 사용
)
processor = AutoProcessor.from_pretrained(model_id)  # 텍스트/이미지 전처리기 로드

/usr/local/lib/python3.11/dist-packages/transformers/models/auto/modeling_auto.py:2199: FutureWarning: The class `AutoModelForVision2Seq` is deprecated and will be removed in v5.0. Please use `AutoModelForImageTextToText` instead.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 5 files:   0%|          | 0/5 [00:00<?, ?it/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/3.86G [00:00<?, ?B/s]

...

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

You have video processor config saved in `preprocessor.json` file which is deprecated. Video processor configs should be saved in their own `video_preprocessor.json` file. You can rename the file or load and save the processor back which renames it automatically. Loading from `preprocessor.json` will be removed in v5.0.


chat_template.json: 0.00B [00:00, ?B/s]

In [25]:
# Preparation for inference
text = processor.apply_chat_template(
    train_dataset[0]["messages"], tokenize=False, add_generation_prompt=False
)
print(text)

<|im_start|>system
당신은 이미지와 제품명(name)으로부터 패션/스타일 정보를 추론하는 분류 모델입니다.<|im_end|>
<|im_start|>user
입력 정보:
- name: Mr.Men Men's Charcoal White T-shirt
- image: [image]

위 정보를 바탕으로, 아래 7가지 key에 대한 값을 JSON 형태로 추론해 주세요:
1) gender
2) masterCategory
3) subCategory
4) season
5) usage
6) baseColour
7) articleType

출력 시 **아래 JSON 예시 형태**를 반드시 지키세요:
{
  "gender": "예시값",
  "masterCategory": "예시값",
  "subCategory": "예시값",
  "season": "예시값",
  "usage": "예시값",
  "baseColour": "예시값",
  "articleType": "예시값"
}

# 예시
{
  "gender": "Men",
  "masterCategory": "Accessories",
  "subCategory": "Eyewear",
  "season": "Winter",
  "usage": "Casual",
  "baseColour": "Blue",
  "articleType": "Sunglasses"
}

# 주의
- 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.
<|vision_start|><|image_pad|><|vision_end|><|im_end|>
<|im_start|>assistant
{"gender": "Men", "masterCategory": "Apparel", "subCategory": "Topwear", "season": "Fall", "usage": "Casual", "baseColour": "Grey", "articleType": "Tshirts"}<|im_end|>



In [26]:
peft_config = LoraConfig(
       # 모델 가중치에 LoRA 업데이트를 적용하는 정도를 조절하는 스케일링 계수
       lora_alpha=128,
       # 과적합을 방지하기 위한 드롭아웃 비율 설정
       lora_dropout=0.05,
       # LoRA의 순위(rank) - 저차원 행렬의 차원을 결정
       r=256,
       # 편향(bias) 업데이트 여부 - 'none'은 편향을 업데이트하지 않음
       bias="none",
       # LoRA를 적용할 대상 모듈들 - 트랜스포머 모델의 주요 투영 레이어들
       target_modules=[
           "q_proj",    # Query 투영 레이어
           "up_proj",   # FFN 상향 투영 레이어
           "o_proj",    # Output 투영 레이어
           "k_proj",    # Key 투영 레이어
           "down_proj", # FFN 하향 투영 레이어
           "gate_proj", # FFN 게이트 투영 레이어
           "v_proj"     # Value 투영 레이어
       ],
       # 작업 유형 지정 - 인과적 언어 모델링(다음 토큰 예측)
       task_type="CAUSAL_LM",
)

In [27]:
args = SFTConfig(
    output_dir="Qwen2-VL-7B-Instruct_LoRA_custom_20250805",           # 저장될 디렉토리와 저장소 ID
    num_train_epochs=2,                      # 학습할 총 에포크 수
    per_device_train_batch_size=16,           # GPU당 배치 크기
    gradient_accumulation_steps=8,           # 그래디언트 누적 스텝 수
    gradient_checkpointing=True,             # 메모리 절약을 위한 체크포인팅
    optim="adamw_torch_fused",               # 최적화기
    logging_steps=10,                        # 로그 기록 주기
    save_strategy="steps",                   # 저장 전략
    save_steps=50,                           # 저장 주기
    bf16=True,                              # bfloat16 사용
    learning_rate=1e-4,                     # 학습률
    max_grad_norm=0.3,                      # 그래디언트 클리핑
    warmup_ratio=0.03,                      # 워밍업 비율
    lr_scheduler_type="constant",           # 고정 학습률
    push_to_hub=False,                      # 허브 업로드 안 함
    remove_unused_columns=False,
    dataset_kwargs={"skip_prepare_dataset": True},
    report_to=None
)

In [28]:
def collate_fn(examples):
   """
   텍스트와 이미지가 포함된 대화 데이터를 모델 학습에 적합한 형태로 변환하는 함수

   Args:
       examples: 각각 "messages" 키를 가진 딕셔너리들의 리스트
                messages는 role(system/user/assistant)과 content를 포함하는 대화 형태

   Returns:
       batch: 모델 학습에 사용할 수 있는 토큰화된 텍스트, 이미지, 라벨이 포함된 배치
   """

   # 1단계: 텍스트 전처리 - 채팅 템플릿 적용
   # 각 예제의 messages를 모델 고유의 채팅 형식으로 변환
   # 모델마다 다른 특수 토큰과 형식을 사용 (예: <|im_start|>, <|system|> 등)
   texts = [processor.apply_chat_template(example["messages"], tokenize=False) for example in examples]

   # 2단계: 이미지 데이터 추출 및 전처리
   # messages에서 이미지 정보를 추출하여 모델이 처리할 수 있는 형태로 변환
   # process_vision_info()는 messages에서 이미지를 찾아 적절한 형태로 변환해주는 함수
   image_inputs = [process_vision_info(example["messages"])[0] for example in examples]

   # 3단계: 텍스트 토크나이징 + 이미지 인코딩
   # 텍스트를 토큰으로 변환하고 이미지를 인코딩하여 하나의 배치로 묶음
   # return_tensors="pt": PyTorch 텐서 형태로 반환
   # padding=True: 배치 내 모든 시퀀스를 같은 길이로 맞춤 (짧은 것은 패딩 토큰으로 채움)
   batch = processor(text=texts, images=image_inputs, return_tensors="pt", padding=True)

   # 4단계: 라벨 생성 (손실 계산용)
   # input_ids를 복사하여 라벨로 사용 (다음 토큰 예측 학습을 위함)
   labels = batch["input_ids"].clone()

   # 5단계: 패딩 토큰 손실 계산에서 제외
   # 패딩된 부분은 실제 데이터가 아니므로 손실 계산에서 제외
   # -100으로 설정하면 CrossEntropyLoss에서 자동으로 무시됨
   labels[labels == processor.tokenizer.pad_token_id] = -100

   # 6단계: 이미지 토큰 손실 계산에서 제외
   # 이미지 토큰은 예측 대상이 아니므로 손실 계산에서 제외
   if isinstance(processor, Qwen2VLProcessor):
       # Qwen2VL 모델에서 사용하는 특수 이미지 토큰들의 ID
       # 151652: 이미지 시작 토큰, 151653: 이미지 종료 토큰, 151655: 이미지 패치 토큰
       image_tokens = [151652, 151653, 151655]
   else:
       # 다른 비전-언어 모델의 이미지 토큰 ID 추출
       image_tokens = [processor.tokenizer.convert_tokens_to_ids(processor.image_token)]

   # 이미지 토큰들을 손실 계산에서 제외 (-100으로 설정)
   for image_token_id in image_tokens:
       labels[labels == image_token_id] = -100

   # 7단계: 최종 배치에 라벨 추가
   # 모델 학습 시 손실 계산에 사용될 라벨을 배치에 추가
   batch["labels"] = labels

   return batch

In [29]:
# 단일 예시 확인
example = train_dataset[0]  # 데이터셋의 첫 번째 아이템
print("단일 예시 데이터:")
print(example)

# collate_fn 테스트 (배치 크기 1로)
batch = collate_fn([example])
print("\n처리된 배치 데이터:")
print("입력 ID 형태:", batch["input_ids"].shape)
print("어텐션 마스크 형태:", batch["attention_mask"].shape)
print("이미지 픽셀 형태:", batch["pixel_values"].shape)
print("레이블 형태:", batch["labels"].shape)

단일 예시 데이터:
{'messages': [{'role': 'system', 'content': [{'type': 'text', 'text': '당신은 이미지와 제품명(name)으로부터 패션/스타일 정보를 추론하는 분류 모델입니다.'}]}, {'role': 'user', 'content': [{'type': 'text', 'text': '입력 정보:\n- name: Mr.Men Men\'s Charcoal White T-shirt\n- image: [image]\n\n위 정보를 바탕으로, 아래 7가지 key에 대한 값을 JSON 형태로 추론해 주세요:\n1) gender\n2) masterCategory\n3) subCategory\n4) season\n5) usage\n6) baseColour\n7) articleType\n\n출력 시 **아래 JSON 예시 형태**를 반드시 지키세요:\n{\n  "gender": "예시값",\n  "masterCategory": "예시값",\n  "subCategory": "예시값",\n  "season": "예시값",\n  "usage": "예시값",\n  "baseColour": "예시값",\n  "articleType": "예시값"\n}\n\n# 예시\n{\n  "gender": "Men",\n  "masterCategory": "Accessories",\n  "subCategory": "Eyewear",\n  "season": "Winter",\n  "usage": "Casual",\n  "baseColour": "Blue",\n  "articleType": "Sunglasses"\n}\n\n# 주의\n- 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.\n'}, {'type': 'image', 'image': <PIL.PngImagePlugin.PngImageFile image mode=RGB size=60x80 at 0x7821AB903F50>}]}, {'role': 'assistant', '

In [30]:
print('입력에 대한 정수 인코딩 결과:')
print(batch["input_ids"][0])

입력에 대한 정수 인코딩 결과:
tensor([151644,   8948,    198,  64795,  82528,  33704,  90667,  21329,  80573,
        138017,  79632,   3153,      8,  42039, 126558,  45104,    101,  92031,
            14, 141274,  32077,  60039,  18411,  57835, 126605,  42905, 128618,
         97929,  54070, 142713,  78952,     13, 151645,    198, 151644,    872,
           198,  43866,  28754,  60039,    510,     12,    829,     25,   4392,
          1321,    268,  11012,    594,   4864,  40465,   5807,    350,  33668,
           198,     12,   2168,     25,    508,   1805,   2533,  80901,  60039,
         18411,  81718, 144059,  42039,     11, 136646,    220,     22,  19969,
         21329,   1376,  19391, 128605,  93668,   4718, 141966,  17380,  57835,
        126605,  33883,  55673,  50302,    510,     16,      8,   9825,    198,
            17,      8,   7341,   6746,    198,     18,      8,   1186,   6746,
           198,     19,      8,   3200,    198,     20,      8,  10431,    198,
            21,      8

In [31]:
print('레이블에 대한 정수 인코딩 결과:')
print(batch["labels"][0])

레이블에 대한 정수 인코딩 결과:
tensor([151644,   8948,    198,  64795,  82528,  33704,  90667,  21329,  80573,
        138017,  79632,   3153,      8,  42039, 126558,  45104,    101,  92031,
            14, 141274,  32077,  60039,  18411,  57835, 126605,  42905, 128618,
         97929,  54070, 142713,  78952,     13, 151645,    198, 151644,    872,
           198,  43866,  28754,  60039,    510,     12,    829,     25,   4392,
          1321,    268,  11012,    594,   4864,  40465,   5807,    350,  33668,
           198,     12,   2168,     25,    508,   1805,   2533,  80901,  60039,
         18411,  81718, 144059,  42039,     11, 136646,    220,     22,  19969,
         21329,   1376,  19391, 128605,  93668,   4718, 141966,  17380,  57835,
        126605,  33883,  55673,  50302,    510,     16,      8,   9825,    198,
            17,      8,   7341,   6746,    198,     18,      8,   1186,   6746,
           198,     19,      8,   3200,    198,     20,      8,  10431,    198,
            21,      

In [32]:
# 토큰 디코딩 예시 (입력 텍스트가 어떻게 변환되었는지 확인)
decoded_text = processor.tokenizer.decode(batch["input_ids"][0])
print("\n디코딩된 텍스트:")
print(decoded_text)


디코딩된 텍스트:
<|im_start|>system
당신은 이미지와 제품명(name)으로부터 패션/스타일 정보를 추론하는 분류 모델입니다.<|im_end|>
<|im_start|>user
입력 정보:
- name: Mr.Men Men's Charcoal White T-shirt
- image: [image]

위 정보를 바탕으로, 아래 7가지 key에 대한 값을 JSON 형태로 추론해 주세요:
1) gender
2) masterCategory
3) subCategory
4) season
5) usage
6) baseColour
7) articleType

출력 시 **아래 JSON 예시 형태**를 반드시 지키세요:
{
  "gender": "예시값",
  "masterCategory": "예시값",
  "subCategory": "예시값",
  "season": "예시값",
  "usage": "예시값",
  "baseColour": "예시값",
  "articleType": "예시값"
}

# 예시
{
  "gender": "Men",
  "masterCategory": "Accessories",
  "subCategory": "Eyewear",
  "season": "Winter",
  "usage": "Casual",
  "baseColour": "Blue",
  "articleType": "Sunglasses"
}

# 주의
- 7개 항목 이외의 정보(텍스트, 문장 등)는 절대 포함하지 마세요.
<|vision_start|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|image_pad|><|vision_end|><|im_end|>
<|im_start|>assistant
{"gender": "Men", "masterCategory": "Apparel", "subCategory": "Topwear", "season": "Fall", "usage": "Casual", "baseCol

In [ ]:
%pip uninstall -y bitsandbytes
%pip uninstall -r requirements.txt
%pip install bitsandbytes==0.37.0


In [ ]:
!pip show bitsandbytes

In [33]:
trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    data_collator=collate_fn,
    peft_config=peft_config,
    processing_class=processor.tokenizer
)

In [34]:
# 학습 시작
trainer.train()   # 모델이 자동으로 허브와 output_dir에 저장됨

# 모델 저장
trainer.save_model()   # 최종 모델을 저장

/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]


/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:92: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torch/utils/checkpoint.py:295: FutureWarning: `torch.cpu.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cpu', args...)` instead.
  with torch.enable_grad(), device_autocast_ctx, torch.cpu.amp.autocast(**ctx.cpu_autocast_kwargs):  # type: ignore[attr-defined]
